https://medium.com/machine-learning-world/feature-extraction-and-similar-image-search-with-opencv-for-newbies-3c59796bf774
    

In [37]:
import warnings
warnings.filterwarnings('ignore')
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import *
from keras.optimizers import SGD, RMSprop
from keras import backend as K

In [38]:
def euclidean_distance(inputs):
    assert len(inputs) == 2, \
        'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    u, v = inputs
    return K.sqrt((K.square(u - v)).sum(axis=1, keepdims=True))


def contrastive_loss(y, d):
    """ Contrastive loss from Hadsell-et-al.'06
        http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """
    margin = 1
    return K.mean(y * K.square(d) + (1 - y) * K.square(K.maximum(margin - d, 0)))


def create_pairs(x, digit_indices):
    """ Positive and negative pair creation.
        Alternates between positive and negative pairs.
    """
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(in_dim):
    """ Base network to be shared (eq. to feature extraction).
    """
    seq = Sequential()
    seq.add(Dense(128, input_shape=(in_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq


def compute_accuracy(predictions, labels):
    """ Compute classification accuracy with a fixed threshold on distances.
    """
    return labels[predictions.ravel() < 0.5].mean()

In [39]:
# the data, shuffled and split between tran and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
in_dim = 784
nb_epoch = 20

In [41]:
X_train.shape

(60000, 784)

In [42]:
y_train.shape

(60000,)

Training Size: 60000 <br>
Training Dimensions: 784 <br>

In [ ]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(10)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)

In [49]:
print(len(digit_indices))
print(len(digit_indices[0]))
print(len(digit_indices[1]))
print(len(digit_indices[2]))
print(len(digit_indices[3]))
print(len(digit_indices[4]))
print(len(digit_indices[5]))
print(len(digit_indices[6]))
print(len(digit_indices[7]))
print(len(digit_indices[8]))
print(len(digit_indices[9]))

10
980
1135
1032
1010
982
892
958
1028
974
1009


In [51]:
tr_pairs.shape

(108400, 2, 784)

In [55]:
tr_y.shape

(108400,)

Training Size: 108400 <br>
Each Training Size has two inputs <br>
Each Input is a 784 Dimensional Vector <br>

In [52]:
digit_indices = [np.where(y_test == i)[0] for i in range(10)]
te_pairs, te_y = create_pairs(X_test, digit_indices)

In [53]:
te_pairs.shape

(17820, 2, 784)

In [57]:
te_y.shape

(17820,)

In [58]:
input1 = Sequential()
input1.add(Layer(input_shape=(in_dim,)))


#base_network = create_base_network(in_dim)

input2 = Sequential()
input2.add(Layer(input_shape=(in_dim,)))

In [21]:
lambda_merge = LambdaMerge([input1, input2], euclidean_distance)


model = Sequential()
model.add(lambda_merge)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y, batch_size=128, nb_epoch=nb_epoch,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

NameError: name 'LambdaMerge' is not defined

In [4]:
# compute final accuracy on training and test sets
pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(pred, tr_y)
pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(pred, te_y)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

 2965504/11490434 [======>.......................] - ETA: 37s

KeyboardInterrupt: 